In [ ]:
1.	import os
2.	from dotenv import load_dotenv
3.	from langchain_openai import ChatOpenAI
4.	from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
5.	from langchain_core.runnables.history import RunnableWithMessageHistory
6.	from langchain_core.chat_history import BaseChatMessageHistory
7.	from langchain.memory import ChatMessageHistory
8.
9.	# LLM configuration
10.	MODEL = "gpt-4o-mini"
11.	TEMPERATURE_PATIENT = 0.8
12.	MAX_TURNS = 20
13.	SESSION_ID = "default"
14.
15.	# Simulation Scenario
16.	SCENARIO = {
17.	    "title": "Chest Pain in a 54-year-old",
18.	    "chief_complaint": "Substernal chest pain for about 2 hours.",
19.	    "background": (
20.	        "54-year-old male, BMI 29. Smokes 15 pack-years, hypertension (poorly controlled). "
21.	        "Father died of a heart attack at 58. Pain began while climbing stairs; "
22.	        "radiates to the left arm; associated with sweating and nausea."
23.	    ),
24.	    "red_flags": "Crushing pain, exertional onset, radiation, diaphoresis; rule out ACS.",
25.	    "persona": "Anxious but cooperative; answers briefly unless asked open questions; may ramble when reassured."
26.	}
27.
28.	SYSTEM_PROMPT_PATIENT = """You are acting as a patient in a clinical interview. Stay fully in character.
29.	Rules:
30.	- Do NOT give clinician-side analysis, diagnoses, or medical advice.
31.	- Answer like a real patient: symptoms, feelings, history; keep details consistent.
32.	- Personality: {persona}
33.	- Scenario:
34.	  - Chief complaint: {cc}
35.	  - Background: {bg}
36.	  - Red flags to reflect if probed: {rf}
37.	- If asked closed questions, be brief. If open questions, be naturally descriptive.
38.	- If asked for test results not yet done, say you haven't had that test.
39.
40.	Only reply as the PATIENT.
41.	"""
42.
43.	# Storing the session history
44.	_store = {}
45.	def get_history(session_id: str) -> BaseChatMessageHistory:
46.	    if session_id not in _store:
47.	        _store[session_id] = ChatMessageHistory()
48.	    return _store[session_id]
49.
50.	def main():
51.	    load_dotenv()
52.	    if not os.getenv("OPENAI_API_KEY"):
53.	        raise RuntimeError("OPENAI_API_KEY not set.")
54.
55.	    patient_llm = ChatOpenAI(model=MODEL, temperature=TEMPERATURE_PATIENT)
56.
57.	    patient_prompt = ChatPromptTemplate.from_messages([
58.	        ("system", SYSTEM_PROMPT_PATIENT),
59.	        MessagesPlaceholder("history"),
60.	        ("human", "{input}")
61.	    ])
62.
63.	    patient_chain_core = patient_prompt | patient_llm
64.	    patient_chain = RunnableWithMessageHistory(
65.	        patient_chain_core,
66.	        get_history,
67.	        input_messages_key="input",
68.	        history_messages_key="history",
69.	    )
70.
71.	    print("\n=== LLM-Simulated Patient  ===")
72.	    print("Scenario: Chest Pain in a 54-year-old")
73.	    print("Type 'end' to finish.\n")
74.	    opening = f"(The patient looks worried.) 'Hi doctor… {SCENARIO['chief_complaint']}'"
75.	    print(f"PATIENT: {opening}")
76.
77.	    get_history(SESSION_ID).add_ai_message(opening)
78.
79.	    turns = 0
80.	    while turns < MAX_TURNS:
81.	        user_msg = input("YOU: ").strip()
82.	        if user_msg.lower() in {"end", "quit", "exit"}:
83.	            break
84.	        if not user_msg:
85.	            continue
86.
87.	        resp = patient_chain.invoke(
88.	            {
89.	                "input": user_msg,
90.	                "persona": SCENARIO["persona"],
91.	                "cc": SCENARIO["chief_complaint"],
92.	                "bg": SCENARIO["background"],
93.	                "rf": SCENARIO["red_flags"],
94.	            },
95.	            config={"configurable": {"session_id": SESSION_ID}},
96.	        )
97.	        print(f"PATIENT: {resp.content.strip()}")
98.	        turns += 1
99.	if __name__ == "__main__":
100.	    main()

In [ ]:
1.	import os, argparse
2.	from datetime import datetime
3.	from dotenv import load_dotenv
4.	from langchain_openai import ChatOpenAI
5.	from langchain_core.prompts import ChatPromptTemplate
6.
7.	DEFAULT_TOPICS = [
8.	   "SELECT basics",
9.	   "WHERE filters (AND/OR, IN, BETWEEN, LIKE)",
10.	   "JOINs (INNER, LEFT, RIGHT, FULL)",
11.	   "GROUP BY and HAVING",
12.	   "ORDER BY and LIMIT/OFFSET",
13.	]
14.
15.	SYSTEM_MSG = """You are an SQL educator. Generate assessment QUESTIONS ONLY (no answers).
16.	Mix query-writing, short-answer, and a few MCQs. Ask clear and real-world questions.
17.	Number the questions under each topic. No explanations or hints."""
18.
19.	TOPIC_TMPL = ChatPromptTemplate.from_template(
20.	   """Topic: {topic}
21.	Generate {n} varied SQL assessment questions (questions only)."""
22.	)
23.
24.	def main():
25.	   load_dotenv()
26.	   key = os.getenv("OPENAI_API_KEY")
27.	   if not key:
28.	       raise RuntimeError("Set OPENAI_API_KEY")
29.
30.	   p = argparse.ArgumentParser()
31.	   p.add_argument("--model", default="gpt-4o-mini")
32.	   p.add_argument("--topics", nargs="*", default=DEFAULT_TOPICS)
33.	   p.add_argument("--n", type=int, default=6, help="Questions per topic")
34.	   p.add_argument("--out", default=None)
35.	   args = p.parse_args()
36.
37.	   llm = ChatOpenAI(model=args.model, temperature=0.4)
38.	   all_text = []
39.
40.	   for t in args.topics:
41.	       msgs = [
42.	           ("system", SYSTEM_MSG),
43.	           ("human", TOPIC_TMPL.format(n=args.n, topic=t)),
44.	       ]
45.	       resp = llm.invoke(msgs).content.strip()
46.	       if not resp.lower().startswith("topic:"):
47.	           resp = f"Topic: {t}\n{resp}"
48.	       all_text.append(resp)
49.
50.	   out_path = args.out or f"sql_questions_{datetime.now():%Y%m%d_%H%M%S}.txt"
51.	   with open(out_path, "w", encoding="utf-8") as f:
52.	       f.write("\n\n".join(all_text))
53.
54.	   print(f" Wrote questions to {out_path}")
55.
56.	if __name__ == "__main__":
57.	   main()

In [ ]:
1.	import os
2.	from datetime import datetime
3.	from dotenv import load_dotenv
4.	from langchain_openai import ChatOpenAI
5.	from langchain_core.prompts import ChatPromptTemplate
6.
7.	TOPIC = "Intro to SQL JOINs"
8.	GRADE = "Undergraduate / Year 1"
9.	DURATION = 60  # minutes
10.	OBJECTIVES = [
11.	   "Explain what SQL JOINs are and why they are used",
12.	   "Differentiate INNER, LEFT, and RIGHT joins",
13.	   "Apply joins in sample queries"
14.	]
15.	ASSESSMENT = "Formative questioning and exit ticket"
16.
17.	SYSTEM_MSG = "You are an experienced educator. Create a concise, classroom-ready lesson plan in Markdown."
18.
19.	PROMPT = ChatPromptTemplate.from_template(
20.	"""Create a lesson plan in Markdown.
21.
22.	Topic: {topic}
23.	Grade/Level: {grade}
24.	Duration: {duration} minutes
25.	Objectives: {objectives}
26.	Assessment: {assessment}
27.
28.	Include:
29.	- Title (H1)
30.	- Overview (2–3 sentences)
31.	- Learning objectives
32.	- Materials
33.	- Timed agenda with teacher + student actions
34.	- Differentiation/supports
35.	- Assessment during and at end
36.	""")
37.
38.	def main():
39.	   load_dotenv()
40.	   if not os.getenv("OPENAI_API_KEY"):
41.	       raise RuntimeError("Set OPENAI_API_KEY")
42.
43.	   llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)
44.	   md = llm.invoke([
45.	       ("system", SYSTEM_MSG),
46.	       ("human", PROMPT.format(
47.	           topic=TOPIC,
48.	           grade=GRADE,
49.	           duration=DURATION,
50.	           objectives="; ".join(OBJECTIVES),
51.	           assessment=ASSESSMENT
52.	       )),
53.	   ]).content.strip()
54.
55.	   out_path = f"lesson_{datetime.now():%Y%m%d_%H%M%S}.md"
56.	   with open(out_path, "w", encoding="utf-8") as f:
57.	       f.write(md)
58.	   print(f" Lesson plan saved to: {out_path}")
59.
60.	if __name__ == "__main__":
61.	   main()
